In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta
import uuid

# Constants
random.seed(42)
flight_date = datetime(2025, 6, 18)
num_flights = 100
num_pilots = 200
num_attendants = 400

aircraft_types = ['A320', 'B737', 'A321', 'CRJ900', 'B787']
destinations = ['LAX', 'ATL', 'JFK', 'DFW', 'DEN', 'PHX', 'SEA', 'MIA', 'SFO', 'CLT']
fare_classes = ['Y', 'M', 'J']
cabins = ['Economy', 'Business', 'First']
statuses = ['On Time', 'Delayed', 'Cancelled']
disruption_reasons = {
    'Cancelled': ['Weather', 'Mechanical Issue', 'Crew Unavailable'],
    'Delayed': ['Late Arrival', 'Crew Delay', 'ATC Congestion']
}
first_names = ['Alex', 'Taylor', 'Jordan', 'Morgan', 'Casey', 'Reese', 'Jamie', 'Avery']
last_names = ['Smith', 'Johnson', 'Lee', 'Brown', 'Garcia', 'Martinez', 'Davis']
gates = [f"{random.choice('ABCDEFGH')}{random.randint(1, 30)}" for _ in range(20)]


In [2]:
# Generate Flights

flights = []
flight_times = []

for _ in range(num_flights):
    dep_minutes = random.randint(0, 1439)
    dep_time = flight_date.replace(hour=0, minute=0) + timedelta(minutes=dep_minutes)
    arr_time = dep_time + timedelta(hours=random.randint(2, 4), minutes=random.randint(0, 59))
    flight_number = f"UA{random.randint(100, 9999)}"
    destination = random.choice(destinations)
    status = random.choices(statuses, weights=[0.75, 0.2, 0.05])[0]
    reason = random.choice(disruption_reasons[status]) if status != 'On Time' else ''
    
    flights.append({
        "FlightNumber": flight_number,
        "Origin": "ORD",
        "Destination": destination,
        "ScheduledDeparture": dep_time,
        "ScheduledArrival": arr_time,
        "Status": status,
        "DisruptionReason": reason,
        "Gate": random.choice(gates),
        "Aircraft": random.choice(aircraft_types),
        "SeatsAvailable": random.randint(0, 20),
        "FareClass": random.choice(fare_classes),
        "Cabin": random.choice(cabins),
        "IsRebookable": status == "Cancelled"
    })
    flight_times.append((flight_number, dep_time, arr_time))

df_flights = pd.DataFrame(flights)


In [3]:
# Generate Passengers

passengers = []
for flight in df_flights.itertuples():
    pax_count = random.randint(50, 180)
    for _ in range(pax_count):
        name = f"{random.choice(first_names)} {random.choice(last_names)}"
        seat = f"{random.randint(1, 30)}{random.choice(['A','B','C','D','E','F'])}"
        pnr = str(uuid.uuid4())[:8].upper()
        passengers.append({
            "PassengerID": str(uuid.uuid4()),
            "Name": name,
            "PNR": pnr,
            "FlightNumber": flight.FlightNumber,
            "Seat": seat,
            "Cabin": flight.Cabin,
            "FareClass": flight.FareClass,
            "CheckedIn": random.choice([True, False])
        })

df_passengers = pd.DataFrame(passengers)


In [4]:
# Generate Crew Roster & Assign with Legality Checks

def generate_name():
    return f"{random.choice(first_names)} {random.choice(last_names)}"

# Initial crew pool
crew_roster = []
for _ in range(num_pilots):
    crew_roster.append({
        "CrewID": str(uuid.uuid4()),
        "Name": generate_name(),
        "Role": random.choice(["Captain", "First Officer"]),
        "LastRestEnd": flight_date - timedelta(hours=random.randint(10, 24)),
        "FlightHoursToday": 0,
        "DutyHoursToday": 0,
        "AssignedFlights": []
    })
for _ in range(num_attendants):
    crew_roster.append({
        "CrewID": str(uuid.uuid4()),
        "Name": generate_name(),
        "Role": "Flight Attendant",
        "LastRestEnd": flight_date - timedelta(hours=random.randint(10, 24)),
        "FlightHoursToday": 0,
        "DutyHoursToday": 0,
        "AssignedFlights": []
    })

# Assign crew with legality
crew_assignments = []
unfilled_flights = []

for fid, dep_time, arr_time in flight_times:
    duration = (arr_time - dep_time).total_seconds() / 3600
    duty_duration = duration + 1.5
    def find_crew(role, count):
        selected = []
        for crew in crew_roster:
            if crew["Role"] == role and \
               dep_time >= crew["LastRestEnd"] + timedelta(hours=10) and \
               crew["FlightHoursToday"] + duration <= 8 and \
               crew["DutyHoursToday"] + duty_duration <= 14:
                selected.append(crew)
                if len(selected) == count:
                    break
        return selected

    pilots = find_crew("Captain", 1) + find_crew("First Officer", 1)
    attendants = find_crew("Flight Attendant", random.randint(3, 6))

    if len(pilots) == 2 and len(attendants) >= 3:
        for crew in pilots + attendants:
            crew["FlightHoursToday"] += duration
            crew["DutyHoursToday"] += duty_duration
            crew["LastRestEnd"] = arr_time
            crew["AssignedFlights"].append(fid)
            crew_assignments.append({
                "CrewID": crew["CrewID"],
                "Name": crew["Name"],
                "Role": crew["Role"],
                "FlightNumber": fid,
                "Certification": random.choice(aircraft_types),
                "FlightHours": round(crew["FlightHoursToday"], 2),
                "DutyHours": round(crew["DutyHoursToday"], 2),
                "Legal": True,
                "ViolationReason": ""
            })
    else:
        unfilled_flights.append(fid)
        for role in ["Captain", "First Officer"] + ["Flight Attendant"] * 3:
            crew_assignments.append({
                "CrewID": None,
                "Name": None,
                "Role": role,
                "FlightNumber": fid,
                "Certification": None,
                "FlightHours": 0,
                "DutyHours": 0,
                "Legal": False,
                "ViolationReason": "Insufficient legal crew"
            })

df_crew = pd.DataFrame(crew_assignments)


In [5]:
# Save to CSV files
df_flights.to_csv("flights.csv", index=False)
df_passengers.to_csv("passengers.csv", index=False)
df_crew.to_csv("crew.csv", index=False)
